## MBA em Ciência de Dados
# Redes Neurais e Arquiteturas Profundas

### <span style="color:darkred">Módulo V - Redes neurais auto-associativas e geradoras</span>


### <span style="color:darkred">Avaliação</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

As respostas devem ser dadas no Moodle, use esse notebook apenas para gerar o código necessário para obter as respostas

---

### Questão 1)

Autoencoders do tipo Overcomplete possuem dimensão latente superior a da entrada. Como garantir o aprendizado sem que haja uma cópia simples dos dados?

(a) Utilizando mais camadas, projetando um Autoencoder Overcomplete Profundo que permita obter uma camada latente com maior qualidade<br>
(b) Substituindo a função de custo de perda ou erro quadrático pela função de custo de entropia cruzada<br>
(d) Utilizando normalização do tipo Batch ou Layer para que os dados sejam modificados ao longo da rede neural, assim evitando que haja uma cópia direta da entrada para o código<br>
(d) Impondo uma restrição na projeção do código que penalize o uso de todas as dimensões do espaço latente e privilegie projeções esparsas para uma dada instância.<br>

---

### Questão 2)

Considere uma arquitetura com dois elementos, organizada na forma encoder-decoder. O primeiro elemento, encoder, aprende a gerar uma camada latente com os parâmetros de distribuições de probabilidade, a partir da qual o segundo componente, decoder amostra exemplos a serem reconstruídos. Esse método é conhecido por:

 (a) Continuous Generalized Autoencoder<br>
 (b) Denoising Overcomplete Autoencoder<br>
 (c) Generative Adversarial Network<br>
 (d) Variational Autoencoder

---
### Questão 3)

Nas redes geradoras adversariais, todos os dados de treinamento pertencem a classe positiva, enquanto que todos os dados gerados a partir de uma amostragem aleatória de uma distribuição pertence à classe negativa. É correto afirmar sobre o método de aprendizado de redes geradoras adversariais (em sua formulação original):

 (a) O modelo discriminador é um classificador, que produz uma única perda utilizada para treinar a rede como um todo. Sendo um classificador o responsável pelo ajuste dos parâmetros, consideramos o aprendizado como supervisionado.<br>
 (b) O modelo gerador produz a perda utilizada para treinar a rede como um todo. O aprendizado é não-supervisionado pelo fato de que os dados do gerador são obtidos a partir de um vetor amostrado a partir de uma distribuição aleatória.<br>
 (c) O modelo discriminador (classificador) produz as perdas utilizadas para treinar a rede em associação com o gerador. Os rótulos são calculados não necessitando de anotação manual, sendo portanto não supervisionado.<br>
 (d) O modelo discriminador produz a perda utilizada para treinar a rede como um todo. O aprendizado é semi-supervisionado já que sabemos o rótulo das classes positivas.<br>
 


---

### Questão 4)

Carregue a base de dados `smarphone_activity_dataset.csv`, conforme abaixo, com uma divisão hold-out utilizando os 80% exemplos iniciais para treinamento e os restantes para teste.

Projete um Autoencoder para produzir uma projeção em 2 dimensões, com as seguintes camadas:
* Entrada (com as dimensões da base de dados)
* Dropout de 0.3
* Camada densa de 2 neurônios (código) e ativação linear
* Camada densa de saída (com as dimensões da base de dados) e ativação tangente hiperbólica

Inicialize as sementes `seed(1)` e `set_seed(2)` antes de instanciar o modelo, compilar e treinar.

Utilize a função de custo mean absolute error (mae), otimizador Adam com taxa 0.01, batchsize 16 e treine por 30 épocas.

Após o treinamento, obtenha um scatterplot do código de treinamento, e analise visualmente a distribuição das classes utilizando os 500 primeiros exemplos `:500`. Podemos identificar grupos com quais misturas de classes?

(a) 2 grupos, classes: (1, 3 e 6); (2, 4 e 5) <br>
(b) 3 grupos, classes: (1 e 3); (2); (4, 5 e 6) <br>
(c) 4 grupos, classes: (1 e 3); (2); (4 e 5); (6)<br>
(d) 4 grupos, classes: (1 e 2); (3); (4); (5 e 6) <br>


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from numpy.random import seed
from tensorflow.random import set_seed

In [2]:
df = pd.read_csv("smartphone_activity_dataset.csv")
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_553,feature_554,feature_555,feature_556,feature_557,feature_558,feature_559,feature_560,feature_561,activity
0,0.289,-0.0203,-0.133,-0.995,-0.983,-0.914,-0.995,-0.983,-0.924,-0.935,...,-0.299,-0.710,-0.1130,0.03040,-0.465,-0.0184,-0.841,0.180,-0.0586,5
1,0.278,-0.0164,-0.124,-0.998,-0.975,-0.960,-0.999,-0.975,-0.958,-0.943,...,-0.595,-0.861,0.0535,-0.00743,-0.733,0.7040,-0.845,0.180,-0.0543,5
2,0.280,-0.0195,-0.113,-0.995,-0.967,-0.979,-0.997,-0.964,-0.977,-0.939,...,-0.391,-0.760,-0.1190,0.17800,0.101,0.8090,-0.849,0.181,-0.0491,5
3,0.279,-0.0262,-0.123,-0.996,-0.983,-0.991,-0.997,-0.983,-0.989,-0.939,...,-0.117,-0.483,-0.0368,-0.01290,0.640,-0.4850,-0.849,0.182,-0.0477,5
4,0.277,-0.0166,-0.115,-0.998,-0.981,-0.990,-0.998,-0.980,-0.990,-0.942,...,-0.351,-0.699,0.1230,0.12300,0.694,-0.6160,-0.848,0.185,-0.0439,5


In [3]:
rotulos = np.array(df['activity'])
features = np.array(df.iloc[:, :-1])

In [4]:
perc_train = 0.8

n_train = int(features.shape[0]*perc_train)
n_test = int(features.shape[0]*(1-perc_train))
print(n_train)
print(n_test)

x_train = features[:n_train,:]
y_train = rotulos[:n_train]

x_test = features[n_train:,:]
y_test = rotulos[n_train:]

8239
2059


In [5]:
### defina autoencoder

---

### Questão 5)

Ainda utilizando a base de dados `smarphone_activity_dataset.csv` com a mesma divisão entre treinamento e teste, projete um Autoencoder profundo para produzir uma projeção em 50 dimensões, com as seguintes camadas:
* Entrada (com as dimensões da base de dados)
* 2 Camadas densas com 128 neurônios, ativação tanh
* Dropout de 0.25
* 1 Camada densa com 50 neurônios, ativação tahn, name='code'
* 2 Camadas densas com 128 neurônios, ativação tanh
* Camada densa de saída (com as dimensões da base de dados), ativação tanh

Inicialize as sementes `seed(1)` e `set_seed(2)` antes de instanciar o modelo, compilar e treinar.

Utilize a função de custo mean squared error (mse), otimizador Adam com taxa 0.001, batchsize 16 e treine por 50 épocas.

Após o treinamento, calcule o valor final da função de custo para os dados de treinamento. Obtenha o código de 50 dimensões para o conjunto de treinamento e de teste. Treine um classificador svm (utilizando o pacote `SVC` do `sklearn`), utilizando parâmetro C=0.5 e `random_state=1`, utilizando o código de treinamento, e calcule a acurácia no código do conjunto de teste.

Os valores observados de MSE de treinamento, e acurácia de classificação SVM no teste estão no intervalo:

(a) MSE =[0.10, 0.25]; Acurácia = [0.70, 0.75] <br>
(b) MSE =[0.10, 0.25]; Acurácia = [0.75, 0.77] <br>
(c) MSE =[0.01, 0.05]; Acurácia = [0.80, 0.86] <br>
(d) MSE =[0.00, 0.04]; Acurácia = [0.88, 0.95] <br>

 **Justificativa**: Ver código abaixo.

In [1]:
## projete e execute autoencoder

In [2]:
## exiba perda de treinamento

In [3]:
## obtenha códigos

In [4]:
## treine e teste classificador